# 10 - Feature Matching Tuning on Local Map Crops

This notebook focuses on finding robust matching settings for localizing train images inside a constant map crop.

What is compared:
- Backends: `SIFT`, `SuperPoint+LightGlue`, optional `LoFTR`
- Filters: raw/gray/shadow/edge/denoise + segmentation-like variants
- Strategies:
  - `direct`: query vs full crop
  - `anchor_roi`: query vs ROI around anchor GT (anchor method)
- Invariance handling: scale + rotation sweep per query

Evaluation signal:
- `raw matches`, `MAGSAC inliers`, `success rate`, center error in crop (px)
- visual checks with 4-column plot:
  - anchor vs crop
  - val vs crop


## Notes

- Position GT for both images is available from train labels.
- Orientation GT is not available directly; estimated orientation from homography is still visualized.
- Start with a small benchmark subset, then increase once settings look promising.


In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from time import perf_counter

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch


In [ ]:
# Dependency checks
try:
    from lightglue import LightGlue, SuperPoint
    from lightglue.utils import rbd
    LIGHTGLUE_AVAILABLE = True
except Exception:
    LIGHTGLUE_AVAILABLE = False

try:
    from kornia.feature import LoFTR
    LOFTR_AVAILABLE = True
except Exception:
    LOFTR_AVAILABLE = False

if not hasattr(cv2, 'SIFT_create'):
    print('Warning: OpenCV SIFT is not available in this build.')

# Paths
CANDIDATE_ROOTS = [Path.cwd(), Path.cwd().parent, Path('.'), Path('..')]
PROJECT_ROOT = None
_seen = set()
for cand in CANDIDATE_ROOTS:
    try:
        root = cand.resolve()
    except Exception:
        continue
    key = str(root)
    if key in _seen:
        continue
    _seen.add(key)
    if (root / 'data' / 'data').exists():
        PROJECT_ROOT = root
        break
if PROJECT_ROOT is None:
    raise FileNotFoundError('Could not find project root containing data/data.')

DATA_ROOT = PROJECT_ROOT / 'data' / 'data'
TRAIN_IMG_DIR = DATA_ROOT / 'train_data' / 'train_images'
TRAIN_POS_CSV = DATA_ROOT / 'train_data' / 'train_pos.csv'
TRAIN_CAM_CSV = DATA_ROOT / 'train_data' / 'train_cam.csv'
MAP_PATH = DATA_ROOT / 'map.png'

# Core config
IMAGE_MAX_SIDE = 1280

# Constant crop around anchor GT
CONST_MAP_CROP_BASE_PX = 900
CONST_MAP_CROP_SCALE = 1.0
CONST_MAP_CROP_MIN_PX = 192
CONST_MAP_CROP_MAX_PX = 1600

# Matching params
MATCH_MIN_CONF = 0.03
MAGSAC_REPROJ_THR = 4.0
MAX_NUM_KEYPOINTS = 4096
MIN_INLIERS_SUCCESS = 6

# SIFT params
SIFT_NFEATURES = 6000
SIFT_RATIO_TEST = 0.85

# LoFTR params
LOFTR_MIN_CONF = 0.10

# Sweep (rotation/scale invariance)
SCALE_SWEEP = [0.16, 0.25, 0.40, 0.60, 0.80, 1.00]
ROT_SWEEP_DEG = [-25.0, -15.0, -8.0, 0.0, 8.0, 15.0, 25.0]

# Anchor ROI strategy
ANCHOR_ROI_FACTOR = 0.60  # ROI size relative to crop size

# Filtering params
CLAHE_CLIP_LIMIT = 2.5
CLAHE_GRID = (8, 8)
EDGE_CANNY_LOW = 50
EDGE_CANNY_HIGH = 150
EDGE_DILATE_KERNEL = 3
EDGE_BLEND_GRAY = 0.70
EDGE_BLEND_EDGE = 0.30

# Benchmark settings
BENCH_MAX_PAIRS = 12
BENCH_BACKENDS = ['superpoint_lightglue', 'sift', 'loftr']
BENCH_FILTERS = [
    'raw_rgb',
    'gray',
    'shadow_clahe_rgb',
    'gray_edge_binary',
    'gray_edge_blend',
    'denoise_edge',
    'seg_non_green_gray',
    'seg_structure_mask',
]
BENCH_STRATEGIES = ['direct', 'anchor_roi']

print('project_root:', PROJECT_ROOT)
print('LightGlue available:', LIGHTGLUE_AVAILABLE)
print('LoFTR available:', LOFTR_AVAILABLE)


In [ ]:
# Load data and build consecutive pairs (id, id+1)
train_pos_df = pd.read_csv(TRAIN_POS_CSV)
train_cam_df = pd.read_csv(TRAIN_CAM_CSV)
train_df = train_cam_df.merge(train_pos_df, on='id', how='inner').copy()
train_df['id'] = train_df['id'].astype(int)
train_df = train_df.sort_values('id').reset_index(drop=True)

map_bgr = cv2.imread(str(MAP_PATH), cv2.IMREAD_COLOR)
if map_bgr is None:
    raise FileNotFoundError(f'Map not found: {MAP_PATH}')
map_rgb = cv2.cvtColor(map_bgr, cv2.COLOR_BGR2RGB)
MAP_H, MAP_W = map_rgb.shape[:2]

pairs = []
for i in range(len(train_df) - 1):
    r0 = train_df.iloc[i]
    r1 = train_df.iloc[i + 1]
    id0 = int(r0['id'])
    id1 = int(r1['id'])
    if id1 != id0 + 1:
        continue
    pairs.append({
        'pair_idx': int(len(pairs)),
        'anchor_id': id0,
        'val_id': id1,
        'anchor_x': float(r0['x_pixel']),
        'anchor_y': float(r0['y_pixel']),
        'val_gt_x': float(r1['x_pixel']),
        'val_gt_y': float(r1['y_pixel']),
    })

pair_df = pd.DataFrame(pairs)
print('pairs:', len(pair_df), 'map:', (MAP_W, MAP_H))
display(pair_df.head(20))


In [ ]:
# Utilities: image loading, crop extraction, transforms
_IMAGE_CACHE: Dict[Tuple[int, Optional[int]], np.ndarray] = {}


def resolve_train_image_path(image_id: int) -> Path:
    stems = [f'{int(image_id):04d}', str(int(image_id))]
    exts = ['.JPG', '.jpg', '.jpeg', '.JPEG', '.png', '.PNG']
    for st in stems:
        for ext in exts:
            p = TRAIN_IMG_DIR / f'{st}{ext}'
            if p.exists():
                return p
    raise FileNotFoundError(f'Image not found for id={image_id} in {TRAIN_IMG_DIR}')


def resize_keep_aspect(img_rgb: np.ndarray, max_side: Optional[int]) -> np.ndarray:
    if max_side is None:
        return img_rgb
    h, w = img_rgb.shape[:2]
    m = max(h, w)
    if m <= int(max_side):
        return img_rgb
    s = float(max_side) / float(m)
    nw = max(32, int(round(w * s)))
    nh = max(32, int(round(h * s)))
    return cv2.resize(img_rgb, (nw, nh), interpolation=cv2.INTER_AREA)


def load_train_image_cached(image_id: int, max_side: Optional[int]) -> np.ndarray:
    key = (int(image_id), max_side)
    if key in _IMAGE_CACHE:
        return _IMAGE_CACHE[key]

    p = resolve_train_image_path(int(image_id))
    bgr = cv2.imread(str(p), cv2.IMREAD_COLOR)
    if bgr is None:
        raise RuntimeError(f'Cannot read image: {p}')
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    rgb = resize_keep_aspect(rgb, max_side=max_side)
    _IMAGE_CACHE[key] = rgb
    return rgb


def resize_rgb(img: np.ndarray, scale: float) -> np.ndarray:
    h, w = img.shape[:2]
    nw = max(8, int(round(w * float(scale))))
    nh = max(8, int(round(h * float(scale))))
    return cv2.resize(img, (nw, nh), interpolation=cv2.INTER_AREA)


def rotate_rgb_keep_size(img: np.ndarray, angle_deg: float) -> np.ndarray:
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w / 2.0, h / 2.0), float(angle_deg), 1.0)
    return cv2.warpAffine(
        img,
        M,
        (w, h),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=(0, 0, 0),
    )


def extract_crop_by_size(map_img: np.ndarray, center_xy: Tuple[float, float], crop_w: int, crop_h: int) -> Tuple[np.ndarray, int, int]:
    cw = int(max(16, crop_w))
    ch = int(max(16, crop_h))

    cx, cy = float(center_xy[0]), float(center_xy[1])
    x0 = int(round(cx - cw / 2.0))
    y0 = int(round(cy - ch / 2.0))
    x0 = int(np.clip(x0, 0, max(0, map_img.shape[1] - cw)))
    y0 = int(np.clip(y0, 0, max(0, map_img.shape[0] - ch)))

    x1 = min(map_img.shape[1], x0 + cw)
    y1 = min(map_img.shape[0], y0 + ch)
    crop = map_img[y0:y1, x0:x1]
    return crop, x0, y0


def pair_to_data(pr: pd.Series) -> Dict[str, object]:
    anchor_xy = (float(pr['anchor_x']), float(pr['anchor_y']))
    val_xy = (float(pr['val_gt_x']), float(pr['val_gt_y']))

    anchor_rgb = load_train_image_cached(int(pr['anchor_id']), max_side=IMAGE_MAX_SIDE)
    val_rgb = load_train_image_cached(int(pr['val_id']), max_side=IMAGE_MAX_SIDE)

    tile_size = int(np.clip(
        round(float(CONST_MAP_CROP_BASE_PX) * float(CONST_MAP_CROP_SCALE)),
        int(CONST_MAP_CROP_MIN_PX),
        min(MAP_W, MAP_H, int(CONST_MAP_CROP_MAX_PX)),
    ))
    crop_rgb, x0, y0 = extract_crop_by_size(map_rgb, center_xy=anchor_xy, crop_w=tile_size, crop_h=tile_size)

    anchor_local = (float(anchor_xy[0] - x0), float(anchor_xy[1] - y0))
    val_local = (float(val_xy[0] - x0), float(val_xy[1] - y0))

    return {
        'pr': pr,
        'anchor_rgb': anchor_rgb,
        'val_rgb': val_rgb,
        'crop_rgb': crop_rgb,
        'crop_origin': (int(x0), int(y0)),
        'anchor_local': anchor_local,
        'val_local': val_local,
    }


In [ ]:
# Filter variants (incl. segmentation-like heuristics)
def _to_rgb_uint8(img: np.ndarray) -> np.ndarray:
    if img.ndim == 2:
        img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_GRAY2RGB)
    if img.dtype != np.uint8:
        img = np.clip(img, 0, 255).astype(np.uint8)
    return img


def _gray(img: np.ndarray) -> np.ndarray:
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)


def _shadow_clahe_rgb(img: np.ndarray, clip=2.5, grid=(8, 8)) -> np.ndarray:
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=float(clip), tileGridSize=tuple(grid))
    l2 = clahe.apply(l)
    return cv2.cvtColor(cv2.merge([l2, a, b]), cv2.COLOR_LAB2RGB)


def _edges(gray: np.ndarray) -> np.ndarray:
    e = cv2.Canny(gray, int(EDGE_CANNY_LOW), int(EDGE_CANNY_HIGH))
    k = int(max(1, EDGE_DILATE_KERNEL))
    if k > 1:
        e = cv2.dilate(e, np.ones((k, k), dtype=np.uint8), iterations=1)
    return e


def f_raw(img):
    return img


def f_gray(img):
    return _gray(img)


def f_shadow(img):
    return _shadow_clahe_rgb(img, clip=CLAHE_CLIP_LIMIT, grid=CLAHE_GRID)


def f_gray_edge_binary(img):
    return _edges(_gray(img))


def f_gray_edge_blend(img):
    g = _gray(img)
    e = _edges(g)
    return cv2.addWeighted(g, float(EDGE_BLEND_GRAY), e, float(EDGE_BLEND_EDGE), 0.0)


def f_denoise_edge(img):
    g = cv2.bilateralFilter(_gray(img), d=7, sigmaColor=50, sigmaSpace=50)
    return _edges(g)


def f_seg_non_green_gray(img):
    # segmentation-like heuristic: suppress green vegetation by HSV mask
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    lower = np.array([30, 25, 20], dtype=np.uint8)
    upper = np.array([95, 255, 255], dtype=np.uint8)
    green_mask = cv2.inRange(hsv, lower, upper)
    keep = cv2.bitwise_not(green_mask)
    g = _gray(img)
    out = cv2.bitwise_and(g, g, mask=keep)
    return out


def f_seg_structure_mask(img):
    # segmentation-like structure map: adaptive threshold + edges
    g = _gray(img)
    thr = cv2.adaptiveThreshold(g, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 35, 2)
    e = _edges(g)
    mask = cv2.bitwise_or(thr, e)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8))
    out = cv2.bitwise_and(g, g, mask=mask)
    return out


FILTER_VARIANTS = {
    'raw_rgb': f_raw,
    'gray': f_gray,
    'shadow_clahe_rgb': f_shadow,
    'gray_edge_binary': f_gray_edge_binary,
    'gray_edge_blend': f_gray_edge_blend,
    'denoise_edge': f_denoise_edge,
    'seg_non_green_gray': f_seg_non_green_gray,
    'seg_structure_mask': f_seg_structure_mask,
}


def apply_filter_variant(name: str, img_rgb: np.ndarray) -> np.ndarray:
    if name not in FILTER_VARIANTS:
        raise KeyError(f'Unknown filter variant: {name}')
    return _to_rgb_uint8(FILTER_VARIANTS[name](img_rgb))


In [ ]:
# Backends: SuperPoint+LightGlue, SIFT, optional LoFTR
@dataclass
class MatchResult:
    k0: np.ndarray
    k1: np.ndarray
    matches: np.ndarray
    conf: np.ndarray


class SuperPointLightGlueBackend:
    def __init__(self, min_conf: float = 0.03, max_num_keypoints: int = 4096, device: Optional[str] = None):
        self.min_conf = float(min_conf)
        self.max_num_keypoints = int(max_num_keypoints)
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.extractor = None
        self.matcher = None

    def _lazy_init(self):
        if self.extractor is not None and self.matcher is not None:
            return
        if not LIGHTGLUE_AVAILABLE:
            raise RuntimeError('LightGlue/SuperPoint is not available.')
        self.extractor = SuperPoint(max_num_keypoints=self.max_num_keypoints).eval().to(self.device)
        self.matcher = LightGlue(features='superpoint').eval().to(self.device)

    def _to_tensor(self, img_rgb: np.ndarray) -> torch.Tensor:
        gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        return (torch.from_numpy(gray).float()[None, None] / 255.0).to(self.device)

    @torch.inference_mode()
    def match(self, img0_rgb: np.ndarray, img1_rgb: np.ndarray, min_conf: Optional[float] = None) -> MatchResult:
        self._lazy_init()
        thr = self.min_conf if min_conf is None else float(min_conf)

        t0 = self._to_tensor(img0_rgb)
        t1 = self._to_tensor(img1_rgb)

        f0 = self.extractor.extract(t0)
        f1 = self.extractor.extract(t1)
        out = self.matcher({'image0': f0, 'image1': f1})
        f0, f1, out = [rbd(x) for x in [f0, f1, out]]

        k0 = f0['keypoints'].detach().cpu().numpy().astype(np.float32)
        k1 = f1['keypoints'].detach().cpu().numpy().astype(np.float32)
        m = out['matches'].detach().cpu().numpy().astype(np.int32)

        if m.size == 0:
            return MatchResult(k0, k1, np.zeros((0, 2), dtype=np.int32), np.zeros((0,), dtype=np.float32))

        if m.ndim != 2:
            m = m.reshape(-1, 2)

        if 'scores' in out:
            conf = out['scores'].detach().cpu().numpy().astype(np.float32)
        else:
            conf = np.ones((m.shape[0],), dtype=np.float32)

        if conf.shape[0] != m.shape[0]:
            conf = np.ones((m.shape[0],), dtype=np.float32)

        keep = conf >= thr
        return MatchResult(k0, k1, m[keep], conf[keep])


class LoFTRBackend:
    def __init__(self, min_conf: float = 0.10, device: Optional[str] = None):
        self.min_conf = float(min_conf)
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = None

    def _lazy_init(self):
        if self.model is not None:
            return
        if not LOFTR_AVAILABLE:
            raise RuntimeError('LoFTR is not available (install kornia).')
        self.model = LoFTR(pretrained='outdoor').eval().to(self.device)

    @torch.inference_mode()
    def match(self, img0_rgb: np.ndarray, img1_rgb: np.ndarray, min_conf: Optional[float] = None) -> MatchResult:
        self._lazy_init()
        thr = self.min_conf if min_conf is None else float(min_conf)

        g0 = cv2.cvtColor(img0_rgb, cv2.COLOR_RGB2GRAY)
        g1 = cv2.cvtColor(img1_rgb, cv2.COLOR_RGB2GRAY)
        t0 = (torch.from_numpy(g0).float()[None, None] / 255.0).to(self.device)
        t1 = (torch.from_numpy(g1).float()[None, None] / 255.0).to(self.device)

        out = self.model({'image0': t0, 'image1': t1})
        if 'keypoints0' not in out or out['keypoints0'].numel() == 0:
            return MatchResult(
                np.zeros((0, 2), dtype=np.float32),
                np.zeros((0, 2), dtype=np.float32),
                np.zeros((0, 2), dtype=np.int32),
                np.zeros((0,), dtype=np.float32),
            )

        k0 = out['keypoints0'].detach().cpu().numpy().astype(np.float32)
        k1 = out['keypoints1'].detach().cpu().numpy().astype(np.float32)
        if 'confidence' in out:
            conf = out['confidence'].detach().cpu().numpy().astype(np.float32)
        else:
            conf = np.ones((k0.shape[0],), dtype=np.float32)

        keep = conf >= thr
        k0 = k0[keep]
        k1 = k1[keep]
        conf = conf[keep]
        n = k0.shape[0]
        m = np.column_stack([np.arange(n), np.arange(n)]).astype(np.int32)
        return MatchResult(k0, k1, m, conf)


def match_sift(img0_rgb: np.ndarray, img1_rgb: np.ndarray) -> MatchResult:
    if not hasattr(cv2, 'SIFT_create'):
        raise RuntimeError('OpenCV build has no SIFT (cv2.SIFT_create missing).')

    g0 = cv2.cvtColor(img0_rgb, cv2.COLOR_RGB2GRAY)
    g1 = cv2.cvtColor(img1_rgb, cv2.COLOR_RGB2GRAY)

    sift = cv2.SIFT_create(nfeatures=int(SIFT_NFEATURES), contrastThreshold=0.01, edgeThreshold=10)
    kp0, d0 = sift.detectAndCompute(g0, None)
    kp1, d1 = sift.detectAndCompute(g1, None)

    if d0 is None or d1 is None or len(kp0) == 0 or len(kp1) == 0:
        return MatchResult(
            np.zeros((0, 2), dtype=np.float32),
            np.zeros((0, 2), dtype=np.float32),
            np.zeros((0, 2), dtype=np.int32),
            np.zeros((0,), dtype=np.float32),
        )

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    knn = bf.knnMatch(d0, d1, k=2)

    good = []
    for pair in knn:
        if len(pair) < 2:
            continue
        m, n = pair
        if m.distance < float(SIFT_RATIO_TEST) * n.distance:
            good.append(m)

    k0 = np.array([k.pt for k in kp0], dtype=np.float32)
    k1 = np.array([k.pt for k in kp1], dtype=np.float32)

    if len(good) == 0:
        return MatchResult(k0, k1, np.zeros((0, 2), dtype=np.int32), np.zeros((0,), dtype=np.float32))

    matches = np.array([[m.queryIdx, m.trainIdx] for m in good], dtype=np.int32)
    conf = np.array([1.0 / (m.distance + 1e-6) for m in good], dtype=np.float32)
    conf /= max(1e-6, float(conf.max()))
    return MatchResult(k0, k1, matches, conf)


SP_LG = SuperPointLightGlueBackend(min_conf=MATCH_MIN_CONF, max_num_keypoints=MAX_NUM_KEYPOINTS)
LOFTR_BK = LoFTRBackend(min_conf=LOFTR_MIN_CONF)


def run_backend_match(backend_name: str, img0_rgb: np.ndarray, img1_rgb: np.ndarray) -> MatchResult:
    if backend_name == 'superpoint_lightglue':
        return SP_LG.match(img0_rgb, img1_rgb, min_conf=MATCH_MIN_CONF)
    if backend_name == 'sift':
        return match_sift(img0_rgb, img1_rgb)
    if backend_name == 'loftr':
        return LOFTR_BK.match(img0_rgb, img1_rgb, min_conf=LOFTR_MIN_CONF)
    raise KeyError(f'Unknown backend: {backend_name}')


In [ ]:
# Geometry, sweep search, scoring, visualization helpers

def magsac_homography(k0: np.ndarray, k1: np.ndarray, matches: np.ndarray, reproj_thr: float = 4.0):
    if matches.shape[0] < 8:
        return None, np.zeros((matches.shape[0],), dtype=bool)

    p0 = k0[matches[:, 0]].astype(np.float32)
    p1 = k1[matches[:, 1]].astype(np.float32)

    H, mask = cv2.findHomography(
        p0,
        p1,
        method=cv2.USAC_MAGSAC,
        ransacReprojThreshold=float(reproj_thr),
        maxIters=10000,
        confidence=0.999,
    )
    if H is None or mask is None:
        return None, np.zeros((matches.shape[0],), dtype=bool)
    return H, mask.ravel().astype(bool)


def project_point_homography(H: np.ndarray, xy: Tuple[float, float]) -> Tuple[float, float]:
    p = np.array([float(xy[0]), float(xy[1]), 1.0], dtype=np.float64)
    q = H @ p
    if abs(float(q[2])) < 1e-12:
        return np.nan, np.nan
    return float(q[0] / q[2]), float(q[1] / q[2])


def estimate_orientation_from_h(H: np.ndarray, q_shape: Tuple[int, int, int], offset_xy: Tuple[float, float] = (0.0, 0.0)) -> float:
    hq, wq = q_shape[:2]
    c = (wq * 0.5, hq * 0.5)
    xaxis = (wq * 0.75, hq * 0.5)
    c2 = project_point_homography(H, c)
    x2 = project_point_homography(H, xaxis)
    if not (np.isfinite(c2[0]) and np.isfinite(c2[1]) and np.isfinite(x2[0]) and np.isfinite(x2[1])):
        return np.nan
    dx = (x2[0] + offset_xy[0]) - (c2[0] + offset_xy[0])
    dy = (x2[1] + offset_xy[1]) - (c2[1] + offset_xy[1])
    if abs(dx) < 1e-12 and abs(dy) < 1e-12:
        return np.nan
    return float(np.degrees(np.arctan2(dy, dx)))


def extract_anchor_roi(ref_crop: np.ndarray, anchor_local: Tuple[float, float], factor: float) -> Tuple[np.ndarray, int, int]:
    ch, cw = ref_crop.shape[:2]
    rw = int(np.clip(round(float(cw) * float(factor)), 96, cw))
    rh = int(np.clip(round(float(ch) * float(factor)), 96, ch))
    return extract_crop_by_size(ref_crop, center_xy=anchor_local, crop_w=rw, crop_h=rh)


def run_match_sweep(
    backend_name: str,
    query_rgb: np.ndarray,
    full_crop_rgb: np.ndarray,
    strategy: str,
    anchor_local: Tuple[float, float],
    gt_local: Tuple[float, float],
    scales: List[float],
    rots_deg: List[float],
) -> Dict[str, object]:
    if strategy not in {'direct', 'anchor_roi'}:
        raise ValueError(f'Unknown strategy: {strategy}')

    if strategy == 'anchor_roi':
        ref_rgb, rx, ry = extract_anchor_roi(full_crop_rgb, anchor_local=anchor_local, factor=float(ANCHOR_ROI_FACTOR))
    else:
        ref_rgb = full_crop_rgb
        rx, ry = 0, 0

    best = None

    for s in scales:
        s = float(s)
        if s <= 0.0:
            continue
        q_s = resize_rgb(query_rgb, s) if abs(s - 1.0) > 1e-6 else query_rgb

        for rdeg in rots_deg:
            rdeg = float(rdeg)
            q_sr = rotate_rgb_keep_size(q_s, rdeg) if abs(rdeg) > 1e-6 else q_s

            try:
                m = run_backend_match(backend_name, q_sr, ref_rgb)
                H, inl = magsac_homography(m.k0, m.k1, m.matches, reproj_thr=MAGSAC_REPROJ_THR)
            except Exception as e:
                m = MatchResult(
                    k0=np.zeros((0, 2), dtype=np.float32),
                    k1=np.zeros((0, 2), dtype=np.float32),
                    matches=np.zeros((0, 2), dtype=np.int32),
                    conf=np.zeros((0,), dtype=np.float32),
                )
                H, inl = None, np.zeros((0,), dtype=bool)
                raw_matches, inliers = 0, 0
                pred_full = (np.nan, np.nan)
                err_px = np.nan
                ori_deg = np.nan
                score = (-1, -1, -1.0)
                cand = {
                    'ok': False,
                    'error': str(e),
                    'query_used': q_sr,
                    'ref_used': ref_rgb,
                    'ref_offset': (int(rx), int(ry)),
                    'scale_used': s,
                    'rot_used_deg': rdeg,
                    'match': m,
                    'H': H,
                    'inlier_mask': inl,
                    'raw_matches': raw_matches,
                    'inliers': inliers,
                    'pred_local_full': pred_full,
                    'ori_deg': ori_deg,
                    'err_px': err_px,
                    'score': score,
                }
            else:
                raw_matches = int(m.matches.shape[0])
                inliers = int(inl.sum()) if inl is not None else 0

                if H is not None and inliers >= int(MIN_INLIERS_SUCCESS):
                    q_center = (float(q_sr.shape[1] * 0.5), float(q_sr.shape[0] * 0.5))
                    px_ref, py_ref = project_point_homography(H, q_center)
                    if np.isfinite(px_ref) and np.isfinite(py_ref):
                        pred_full = (float(px_ref + rx), float(py_ref + ry))
                        err_px = float(np.hypot(pred_full[0] - gt_local[0], pred_full[1] - gt_local[1]))
                    else:
                        pred_full = (np.nan, np.nan)
                        err_px = np.nan
                    ori_deg = estimate_orientation_from_h(H, q_sr.shape, offset_xy=(float(rx), float(ry)))
                else:
                    pred_full = (np.nan, np.nan)
                    err_px = np.nan
                    ori_deg = np.nan

                conf_mean = float(m.conf.mean()) if m.conf.shape[0] > 0 else 0.0
                score = (inliers, raw_matches, conf_mean)
                cand = {
                    'ok': True,
                    'error': '',
                    'query_used': q_sr,
                    'ref_used': ref_rgb,
                    'ref_offset': (int(rx), int(ry)),
                    'scale_used': s,
                    'rot_used_deg': rdeg,
                    'match': m,
                    'H': H,
                    'inlier_mask': inl,
                    'raw_matches': raw_matches,
                    'inliers': inliers,
                    'pred_local_full': pred_full,
                    'ori_deg': ori_deg,
                    'err_px': err_px,
                    'score': score,
                }

            if best is None or cand['score'] > best['score']:
                best = cand

    if best is None:
        best = {
            'ok': False,
            'error': 'no candidate',
            'query_used': query_rgb,
            'ref_used': full_crop_rgb,
            'ref_offset': (0, 0),
            'scale_used': 1.0,
            'rot_used_deg': 0.0,
            'match': MatchResult(
                k0=np.zeros((0, 2), dtype=np.float32),
                k1=np.zeros((0, 2), dtype=np.float32),
                matches=np.zeros((0, 2), dtype=np.int32),
                conf=np.zeros((0,), dtype=np.float32),
            ),
            'H': None,
            'inlier_mask': np.zeros((0,), dtype=bool),
            'raw_matches': 0,
            'inliers': 0,
            'pred_local_full': (np.nan, np.nan),
            'ori_deg': np.nan,
            'err_px': np.nan,
            'score': (-1, -1, -1.0),
        }

    return best


def evaluate_pair_one_setting(
    pr: pd.Series,
    backend_name: str,
    filter_name: str,
    strategy: str,
) -> Dict[str, object]:
    data = pair_to_data(pr)

    anchor_f = apply_filter_variant(filter_name, data['anchor_rgb'])
    val_f = apply_filter_variant(filter_name, data['val_rgb'])
    crop_f = apply_filter_variant(filter_name, data['crop_rgb'])

    anchor_res = run_match_sweep(
        backend_name=backend_name,
        query_rgb=anchor_f,
        full_crop_rgb=crop_f,
        strategy=strategy,
        anchor_local=data['anchor_local'],
        gt_local=data['anchor_local'],
        scales=SCALE_SWEEP,
        rots_deg=ROT_SWEEP_DEG,
    )

    val_res = run_match_sweep(
        backend_name=backend_name,
        query_rgb=val_f,
        full_crop_rgb=crop_f,
        strategy=strategy,
        anchor_local=data['anchor_local'],
        gt_local=data['val_local'],
        scales=SCALE_SWEEP,
        rots_deg=ROT_SWEEP_DEG,
    )

    return {
        'pr': pr,
        'data': data,
        'backend': backend_name,
        'filter': filter_name,
        'strategy': strategy,
        'anchor_res': anchor_res,
        'val_res': val_res,
        'anchor_f': anchor_f,
        'val_f': val_f,
        'crop_f': crop_f,
    }


def _pick_draw_idx(match: MatchResult, inl: np.ndarray, max_draw: int = 120, seed: int = 7):
    n = int(match.matches.shape[0])
    if n == 0:
        return np.zeros((0,), dtype=np.int32)

    if inl is not None and inl.shape[0] == n and np.any(inl):
        idx = np.where(inl.astype(bool))[0]
    else:
        idx = np.arange(n, dtype=np.int32)

    if idx.size <= max_draw:
        return idx

    if match.conf is not None and match.conf.shape[0] == n:
        order = np.argsort(-match.conf[idx])[:max_draw]
        return idx[order]

    rng = np.random.default_rng(int(seed))
    return np.sort(rng.choice(idx, size=max_draw, replace=False))


def draw_4col_inspection(res: Dict[str, object], max_draw: int = 120, seed: int = 7):
    # [anchor|crop] [val|crop]
    a = _to_rgb_uint8(res['anchor_res']['query_used'])
    v = _to_rgb_uint8(res['val_res']['query_used'])
    c = _to_rgb_uint8(res['crop_f'])

    h = max(a.shape[0], c.shape[0], v.shape[0], c.shape[0])
    w1, w2, w3, w4 = a.shape[1], c.shape[1], v.shape[1], c.shape[1]
    W = w1 + w2 + w3 + w4
    canvas = np.zeros((h, W, 3), dtype=np.uint8)

    def put(im, x):
        y = (h - im.shape[0]) // 2
        canvas[y:y + im.shape[0], x:x + im.shape[1]] = im
        return (x, y, im.shape[1], im.shape[0])

    p1 = put(a, 0)
    p2 = put(c, w1)
    p3 = put(v, w1 + w2)
    p4 = put(c, w1 + w2 + w3)

    fig, ax = plt.subplots(1, 1, figsize=(30, 6))
    ax.imshow(canvas)
    ax.axis('off')

    # separators
    for xs in [p2[0] - 0.5, p3[0] - 0.5, p4[0] - 0.5]:
        ax.axvline(xs, color='white', linewidth=1.0, alpha=0.6)

    # anchor->crop lines
    ar = res['anchor_res']
    am = ar['match']
    a_idx = _pick_draw_idx(am, ar['inlier_mask'], max_draw=max_draw, seed=seed)
    x1, y1, _, _ = p1
    x2, y2, _, _ = p2
    offx_a, offy_a = ar['ref_offset']

    for i in a_idx:
        ii = int(i)
        qi = int(am.matches[ii, 0])
        ri = int(am.matches[ii, 1])
        q = am.k0[qi]
        r = am.k1[ri]
        ok_inl = bool(ar['inlier_mask'][ii]) if ar['inlier_mask'] is not None and ii < len(ar['inlier_mask']) else False
        ax.plot(
            [float(q[0] + x1), float(r[0] + offx_a + x2)],
            [float(q[1] + y1), float(r[1] + offy_a + y2)],
            color='cyan',
            linewidth=1.2 if ok_inl else 0.6,
            alpha=0.9 if ok_inl else 0.25,
        )

    # val->crop lines
    vr = res['val_res']
    vm = vr['match']
    v_idx = _pick_draw_idx(vm, vr['inlier_mask'], max_draw=max_draw, seed=seed)
    x3, y3, _, _ = p3
    x4, y4, _, _ = p4
    offx_v, offy_v = vr['ref_offset']

    for i in v_idx:
        ii = int(i)
        qi = int(vm.matches[ii, 0])
        ri = int(vm.matches[ii, 1])
        q = vm.k0[qi]
        r = vm.k1[ri]
        ok_inl = bool(vr['inlier_mask'][ii]) if vr['inlier_mask'] is not None and ii < len(vr['inlier_mask']) else False
        ax.plot(
            [float(q[0] + x3), float(r[0] + offx_v + x4)],
            [float(q[1] + y3), float(r[1] + offy_v + y4)],
            color='orange',
            linewidth=1.2 if ok_inl else 0.6,
            alpha=0.9 if ok_inl else 0.25,
        )

    # GT + predicted centers on crop columns
    al = res['data']['anchor_local']
    vl = res['data']['val_local']

    # left crop
    ax.scatter([al[0] + x2], [al[1] + y2], s=30, c='red')
    ax.scatter([vl[0] + x2], [vl[1] + y2], s=28, c='cyan')
    if np.isfinite(ar['pred_local_full'][0]) and np.isfinite(ar['pred_local_full'][1]):
        ax.scatter([ar['pred_local_full'][0] + x2], [ar['pred_local_full'][1] + y2], s=55, c='yellow', marker='*')

    # right crop
    ax.scatter([al[0] + x4], [al[1] + y4], s=30, c='red')
    ax.scatter([vl[0] + x4], [vl[1] + y4], s=28, c='cyan')
    if np.isfinite(vr['pred_local_full'][0]) and np.isfinite(vr['pred_local_full'][1]):
        ax.scatter([vr['pred_local_full'][0] + x4], [vr['pred_local_full'][1] + y4], s=55, c='yellow', marker='*')

    ax.set_title(
        f"{res['filter']} | {res['backend']} | {res['strategy']} | "
        f"A raw={ar['raw_matches']} inl={ar['inliers']} err={ar['err_px']:.1f} | "
        f"V raw={vr['raw_matches']} inl={vr['inliers']} err={vr['err_px']:.1f}"
    )
    plt.tight_layout()
    plt.show()


In [ ]:
# 4-column comparison with SuperPoint+LightGlue matches (inspector)

PAIR_ANCHOR_ID = 13
PAIR_VAL_ID = 14
PREVIEW_PAIR_INDEX = 0

INSPECT_BACKEND = 'superpoint_lightglue'   # 'superpoint_lightglue' | 'sift' | 'loftr'
INSPECT_FILTER = 'gray'                    # key from FILTER_VARIANTS
INSPECT_STRATEGY = 'direct'                # 'direct' | 'anchor_roi'

# pick pair
sel = pair_df[
    (pair_df['anchor_id'].astype(int) == int(PAIR_ANCHOR_ID)) &
    (pair_df['val_id'].astype(int) == int(PAIR_VAL_ID))
]
if len(sel) == 0:
    idx = int(np.clip(PREVIEW_PAIR_INDEX, 0, len(pair_df) - 1))
    pr = pair_df.iloc[idx]
    print(f'Pair {PAIR_ANCHOR_ID}->{PAIR_VAL_ID} not found, using pair index {idx}.')
else:
    pr = sel.iloc[0]

if INSPECT_BACKEND == 'loftr' and not LOFTR_AVAILABLE:
    raise RuntimeError('LoFTR backend selected but kornia/LoFTR is not available.')
if INSPECT_BACKEND == 'superpoint_lightglue' and not LIGHTGLUE_AVAILABLE:
    raise RuntimeError('SuperPoint+LightGlue selected but lightglue is not available.')

res = evaluate_pair_one_setting(
    pr=pr,
    backend_name=INSPECT_BACKEND,
    filter_name=INSPECT_FILTER,
    strategy=INSPECT_STRATEGY,
)

draw_4col_inspection(res, max_draw=120, seed=7)


In [ ]:
# Benchmark: methods x filters x strategies on a subset of pairs

active_backends = []
for b in BENCH_BACKENDS:
    if b == 'superpoint_lightglue' and not LIGHTGLUE_AVAILABLE:
        continue
    if b == 'loftr' and not LOFTR_AVAILABLE:
        continue
    active_backends.append(b)

if len(active_backends) == 0:
    raise RuntimeError('No backend available for benchmark.')

n_pairs = int(min(BENCH_MAX_PAIRS, len(pair_df)))
sub_pairs = pair_df.head(n_pairs)

rows = []
cache = {}

t0 = perf_counter()
for k, pr in enumerate(sub_pairs.itertuples(index=False), 1):
    pr_s = pd.Series(pr._asdict())
    for backend in active_backends:
        for filt in BENCH_FILTERS:
            if filt not in FILTER_VARIANTS:
                continue
            for strat in BENCH_STRATEGIES:
                out = evaluate_pair_one_setting(
                    pr=pr_s,
                    backend_name=backend,
                    filter_name=filt,
                    strategy=strat,
                )

                ar = out['anchor_res']
                vr = out['val_res']

                rows.append({
                    'pair_idx': int(pr_s['pair_idx']),
                    'anchor_id': int(pr_s['anchor_id']),
                    'val_id': int(pr_s['val_id']),
                    'backend': backend,
                    'filter': filt,
                    'strategy': strat,
                    'anchor_raw': int(ar['raw_matches']),
                    'anchor_inliers': int(ar['inliers']),
                    'anchor_err_px': float(ar['err_px']) if np.isfinite(ar['err_px']) else np.nan,
                    'anchor_success': bool(ar['inliers'] >= MIN_INLIERS_SUCCESS),
                    'anchor_scale': float(ar['scale_used']),
                    'anchor_rot': float(ar['rot_used_deg']),
                    'val_raw': int(vr['raw_matches']),
                    'val_inliers': int(vr['inliers']),
                    'val_err_px': float(vr['err_px']) if np.isfinite(vr['err_px']) else np.nan,
                    'val_success': bool(vr['inliers'] >= MIN_INLIERS_SUCCESS),
                    'val_scale': float(vr['scale_used']),
                    'val_rot': float(vr['rot_used_deg']),
                })

                cache[(int(pr_s['pair_idx']), backend, filt, strat)] = out

    if k % 2 == 0:
        print(f'processed pairs: {k}/{n_pairs}')

bench_detail_df = pd.DataFrame(rows)
print('benchmark rows:', len(bench_detail_df), 'runtime_s:', f'{(perf_counter()-t0):.1f}')
display(bench_detail_df.head(20))


In [ ]:
# Aggregate ranking + top visualizations
if len(bench_detail_df) == 0:
    raise RuntimeError('No benchmark data available.')

agg = (
    bench_detail_df
    .groupby(['backend', 'filter', 'strategy'], as_index=False)
    .agg(
        n_pairs=('pair_idx', 'count'),
        anchor_raw_mean=('anchor_raw', 'mean'),
        anchor_inl_mean=('anchor_inliers', 'mean'),
        anchor_success_rate=('anchor_success', 'mean'),
        anchor_err_median=('anchor_err_px', 'median'),
        val_raw_mean=('val_raw', 'mean'),
        val_inl_mean=('val_inliers', 'mean'),
        val_success_rate=('val_success', 'mean'),
        val_err_median=('val_err_px', 'median'),
    )
)

# score prioritizes robust val matching
agg['score'] = (
    100.0 * agg['val_success_rate']
    + 2.0 * agg['val_inl_mean']
    - 0.20 * agg['val_err_median'].fillna(999.0)
)

agg = agg.sort_values('score', ascending=False).reset_index(drop=True)

display(agg.head(30))
print('Pivot val_inl_mean:')
display(agg.pivot_table(index='filter', columns='backend', values='val_inl_mean', aggfunc='max'))

# Visualize top-k settings on chosen pair
TOP_K_VIS = 5
VIS_PAIR_ANCHOR_ID = 13
VIS_PAIR_VAL_ID = 14
VIS_PAIR_INDEX_FALLBACK = 0

sel = pair_df[
    (pair_df['anchor_id'].astype(int) == int(VIS_PAIR_ANCHOR_ID)) &
    (pair_df['val_id'].astype(int) == int(VIS_PAIR_VAL_ID))
]
if len(sel) == 0:
    idx = int(np.clip(VIS_PAIR_INDEX_FALLBACK, 0, len(pair_df) - 1))
    pr = pair_df.iloc[idx]
else:
    pr = sel.iloc[0]
pair_idx = int(pr['pair_idx'])

show_rows = agg.head(int(min(TOP_K_VIS, len(agg))))
for rr in show_rows.itertuples(index=False):
    key = (pair_idx, rr.backend, rr.filter, rr.strategy)
    if key not in cache:
        out = evaluate_pair_one_setting(
            pr=pr,
            backend_name=rr.backend,
            filter_name=rr.filter,
            strategy=rr.strategy,
        )
    else:
        out = cache[key]
    draw_4col_inspection(out, max_draw=120, seed=7)
